In [2]:
import os
import openai
import weave
weave.init('together-weave')


Logged in as Weights & Biases user: swarajraibagi.
View Weave data at https://wandb.ai/swarajraibagi/together-weave/weave


In [ ]:


system_content = "You are a travel agent. Be descriptive and helpful."
user_content = "Tell me about San Francisco"

client = openai.OpenAI(
    api_key=os.getenv("OPENROUTER_API_KEY"),
    base_url=os.getenv("OPENROUTER_BASE_URL"),
)
chat_completion = client.chat.completions.create(
    model="microsoft/phi-3-mini-128k-instruct:free",
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
    temperature=0.7,
    max_tokens=1024,
)
response = chat_completion.choices[0].message.content
print("Model response:\n", response)

In [2]:
def make_open_ai_request(model, messages, temperature=0.7, max_tokens=1024):
    client = openai.OpenAI(
        api_key=os.getenv("OPENROUTER_API_KEY"),
        base_url=os.getenv("OPENROUTER_BASE_URL"),
    )
    for attempt in range(3):
        try:
            chat_completion = client.chat.completions.create(
                model=model,
                messages=messages,
                temperature=temperature,
                max_tokens=max_tokens,
            )
            return chat_completion
        except Exception as e:
            if attempt == 2:  # Last attempt
                raise
            print(f"Attempt {attempt + 1} failed. Retrying...")
    raise Exception("Failed to make OpenAI request after 3 attempts")

make_open_ai_request(
    model='openai/chatgpt-4o-latest',
    messages=[
        {"role": "system", "content": system_content},
        {"role": "user", "content": user_content},
    ],
)

🍩 https://wandb.ai/swarajraibagi/together-weave/r/call/0192171e-5419-7ba3-91df-920024e18a87


ChatCompletion(id='gen-RJeYRHKoRd4JoscMXkbCiuokPjsH', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content="San Francisco is a vibrant and diverse city located in Northern California, known for its iconic landmarks, rich history, and dynamic culture. It’s a city that packs a lot into its compact 49 square miles, offering something for every type of traveler. Let me walk you through some of its highlights:\n\n### **1. Iconic Landmarks:**\n- **Golden Gate Bridge**: This world-famous suspension bridge, with its striking International Orange color, is one of the most photographed landmarks in the world. You can walk or bike across it for stunning views of the bay and the city skyline.\n- **Alcatraz Island**: Once a notorious federal prison, Alcatraz is now a popular tourist destination. A ferry ride takes you to the island where you can explore the prison and learn about its infamous inmates, including Al Capone. The audio tour is highly rec

In [4]:
from src.evaluation_suite import EvaluationSuiteSetupConfig, SetupExample


example_setup = EvaluationSuiteSetupConfig(
    system_prompt="""You are a customer support agent tasked with writing the first response to a user filing a support issue. Follow these guidelines:

For content, begin with a warm greeting and thank the user for reaching out and acknowledge the issue they've reported.

For format, limit the response to 1 short paragraph.

For tone, maintain a professional yet friendly tone throughout.

Use the user's name if provided in their inquiry.

For additional rules, do not provide specific solutions in this first response.
Always end with an open-ended question to encourage further dialogue.
""",
    examples=[
        SetupExample(
            input="My shipment is delayed and I need it urgently.",
            output="Hello [User], Thank you for contacting our support team regarding your delayed shipment. I understand that this delay is causing you concern, especially given the urgency of your need. I sincerely apologize for any inconvenience this may be causing you. I want to assure you that I'm here to help and will do my best to address this situation as quickly as possible. Your satisfaction is important to us, and we take shipping delays very seriously. To better assist you, could you please provide me with your order number and the expected delivery date that was originally given to you? This information will help me investigate the status of your shipment more effectively."),
    ]
)


In [50]:
import requests
import json

# API endpoint URLs
setup_url = "http://localhost:8000/set_up_task"
run_evaluations_url = "http://localhost:8000/run_evaluations"

# Set up task
setup_response = requests.post(
    setup_url,
    json={
        "system_prompt": example_setup.system_prompt,
        "examples": [
            {"input": example.input, "output": example.output}
            for example in example_setup.examples
        ]
    }
)

if setup_response.status_code == 200:
    evaluation_suite = setup_response.json()
    print("Task setup successful.")
    print(json.dumps(evaluation_suite, indent=2))
else:
    print(f"Task setup failed with status code: {setup_response.status_code}")
    print(setup_response.text)


Task setup failed with status code: 400
{"detail":"2 validation errors for AssertionWrapper\nassertion.LLMAssertion\n  Input should be a valid dictionary or instance of LLMAssertion [type=model_type, input_value=WeaveList([LLMAssertion(t...evaluation_type='llm')]), input_type=WeaveList]\n    For further information visit https://errors.pydantic.dev/2.9/v/model_type\nassertion.PythonAssertion\n  Input should be a valid dictionary or instance of PythonAssertion [type=model_type, input_value=WeaveList([LLMAssertion(t...evaluation_type='llm')]), input_type=WeaveList]\n    For further information visit https://errors.pydantic.dev/2.9/v/model_type"}


In [49]:
from src.evaluation_suite import EvaluationSuite


EvaluationSuite(**evaluation_suite)

ValidationError: 7 validation errors for EvaluationSuite
assertions.Warm Greeting and Appreciation
  Extra inputs are not permitted [type=extra_forbidden, input_value={'test_name': 'check_warm...evaluation_type': 'llm'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden
assertions.Issue Acknowledgment
  Extra inputs are not permitted [type=extra_forbidden, input_value={'test_name': 'issue_ackn...evaluation_type': 'llm'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden
assertions.Conciseness
  Extra inputs are not permitted [type=extra_forbidden, input_value={'test_name': 'test_conci...luation_type': 'python'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/extra_forbidden
assertions.Professional and Friendly Tone
  Extra inputs are not permitted [type=extra_forbidden, input_value={'test_name': 'profession...evaluation_type': 'llm'}, input_type=di

In [45]:
# Run evaluations
# Note: This is a placeholder as we don't have the actual testcase and criterion_assertion_map
# You would need to extract these from the evaluation_suite response
run_evaluations_response = requests.post(
    run_evaluations_url,
    json={
        "testcase": {
            "input": "My order hasn't arrived yet. It's been a week.",
            "output": "Hello there! I'm sorry to hear that your order hasn't arrived yet. That must be frustrating. I'd like to help you track down your package. Could you please provide me with your order number so I can look into this for you?"
        },
        "evaluation_suite": evaluation_suite
    }
)

if run_evaluations_response.status_code == 200:
    evaluation_results = run_evaluations_response.json()
    print("Evaluations run successfully.")
    print(json.dumps(evaluation_results, indent=2))
else:
    print(f"Evaluations failed with status code: {run_evaluations_response.status_code}")
    print(run_evaluations_response.text)

Evaluations failed with status code: 422
{"detail":[{"type":"model_attributes_type","loc":["body","evaluation_suite","assertions","name","LLMAssertion"],"msg":"Input should be a valid dictionary or object to extract fields from","input":null,"url":"https://errors.pydantic.dev/2.9/v/model_attributes_type"},{"type":"model_attributes_type","loc":["body","evaluation_suite","assertions","name","PythonAssertion"],"msg":"Input should be a valid dictionary or object to extract fields from","input":null,"url":"https://errors.pydantic.dev/2.9/v/model_attributes_type"},{"type":"model_attributes_type","loc":["body","evaluation_suite","assertions","description","LLMAssertion"],"msg":"Input should be a valid dictionary or object to extract fields from","input":null,"url":"https://errors.pydantic.dev/2.9/v/model_attributes_type"},{"type":"model_attributes_type","loc":["body","evaluation_suite","assertions","description","PythonAssertion"],"msg":"Input should be a valid dictionary or object to extract

In [4]:
{"detail":[{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Warm Greeting"],"msg":"Extra inputs are not permitted","input":[{"test_name":"warm_greeting_presence","text":"Evaluate if the response starts with a warm greeting. A warm greeting should include a friendly salutation (e.g., 'Hello', 'Hi') and should express a positive tone to help de-escalate any initial frustration from the user. If the response begins with such a warm greeting, return 'PASS'. Otherwise, return 'FAIL'.","evaluation_type":"llm"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"},{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Acknowledgment of Issue"],"msg":"Extra inputs are not permitted","input":[{"test_name":"acknowledgment_of_issue","text":"Evaluate whether the response explicitly acknowledges the user's reported issue. Look for phrases or sentences in the response that directly refer to the issue, such as a delayed shipment, in a way that shows clear understanding and acknowledgment of the user's situation. Return 'PASS' if the acknowledgment is present and clear; otherwise, return 'FAIL'.","evaluation_type":"llm"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"},{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Professional Yet Friendly Tone"],"msg":"Extra inputs are not permitted","input":[{"test_name":"professional_yet_friendly_tone","text":"Evaluate whether the LLM response maintains a professional yet friendly tone. The response should be respectful, empathetic, and polite, while also being approachable and warm. Consider whether the language used makes the user feel valued and respected. Return 'PASS' if the response maintains this balance, otherwise return 'FAIL'.","evaluation_type":"llm"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"},{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Limitation to One Short Paragraph"],"msg":"Extra inputs are not permitted","input":[{"test_name":"test_response_limited_to_one_paragraph","code":"def test_response_limited_to_one_paragraph(self):\n    # Ensure the response is limited to one paragraph\n    response_text = self.output['text']\n    # Splitting the text by new lines to count paragraphs\n    paragraphs = response_text.strip().split('\\n')\n    # Check that there is only one non-empty paragraph\n    self.assertEqual(sum(1 for paragraph in paragraphs if paragraph.strip()), 1, \"The response should be limited to one short paragraph.\")","evaluation_type":"python"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"},{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Avoiding Specific Solutions"],"msg":"Extra inputs are not permitted","input":[{"test_name":"avoiding_specific_solutions","text":"Evaluate the response provided and determine if it avoids giving specific solutions regarding the user's issue. The response should focus on acknowledging the user's concern and gathering more information without suggesting concrete steps or solutions in the first reply. If the response refrains from offering specific solutions and instead asks for additional information to proceed, mark it as 'PASS'. Otherwise, mark it as 'FAIL'.","evaluation_type":"llm"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"},{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Using User's Name"],"msg":"Extra inputs are not permitted","input":[{"test_name":"test_use_of_user_name","code":"def test_use_of_user_name(self):\n    # Check if the user's name is included\n    user_name_placeholder = \"[User]\"\n    output_text = self.output.get('Output', '')\n    self.assertIn(user_name_placeholder, output_text, \"The response should include the user's name to add a personal touch.\")","evaluation_type":"python"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"},{"type":"extra_forbidden","loc":["body","criterion_assertion_map","Ending with Open-Ended Question"],"msg":"Extra inputs are not permitted","input":[{"test_name":"evaluate_ending_with_open_ended_question","text":"Given the LLM's response to a customer support inquiry, evaluate if the response concludes with an open-ended question designed to encourage further dialogue. The question should solicit additional information or engagement from the user rather than a simple 'yes' or 'no' answer. Return 'PASS' if the response ends in an open-ended question that invites user interaction and aligns with this criterion. Otherwise, return 'FAIL'.","evaluation_type":"llm"}],"url":"https://errors.pydantic.dev/2.9/v/extra_forbidden"}]}

{'suite_description': 'The task is to generate an initial response to a customer support inquiry. The response must include a warm greeting, acknowledgment of the issue, and an open-ended question for further engagement. The response should be professionally friendly, concise, and should not offer specific solutions in this initial message.',
 'evaluation_criteria': [{'criterion': 'Warm Greeting',
   'explaination': 'The response should start with a warm greeting to make the customer feel valued and acknowledged.',
   'evaluation_method': 'llm'},
  {'criterion': 'Acknowledgement of the Issue',
   'explaination': "Clearly acknowledging the customer's issue shows empathy and assures the customer that their concern is being taken seriously.",
   'evaluation_method': 'llm'},
  {'criterion': 'Professional and Friendly Tone',
   'explaination': 'A professional yet friendly tone is essential to ensure the customer feels comfortable and respected.',
   'evaluation_method': 'llm'},
  {'criterio